In [1]:
%matplotlib widget

In [2]:
from functools import partial

In [3]:
from libschrodinger import *

In [4]:
import numpy as np

In [5]:
import pandas as pd

In [6]:
from pathlib import Path

In [7]:
from typing import Dict

In [8]:
caseName : str = ""

In [9]:
def diagonalConstantWidthPotential(
                position : MeshGrid, 
                potentialRatios : List[float], 
                regionLengthRatios : List[float], 
                potentialHeight : float, 
                regionWidthsRatio : float, 
                yPositionRatio : float, 
                math = np
        ): 
    baseX = math.min(position.y)
    yExtent : float = math.abs(math.max(position.y) - baseX)
    yPosition = (yExtent * yPositionRatio) + baseY
    width = yExtent * regionWidthsRatio
    groundPotential = 0.0 * position.y
    fullWidthPotentials = constantPotentials(
                position, 
                potentialRatios, 
                regionLengthRatios, 
                potentialHeight, 
                regionWidthsRatio, 
                math
        )
    return math.where(
            (position.y < yPosition) | (position.y > width), 
            groundPotential, 
            fullWidthPotentials
        )


In [10]:
math = np
spatialStep : float = .02
length : float = 1
temporalStep : float = (spatialStep ** 2) / 4
courantNumber : float = 1.0
pointCount : int = int(length / spatialStep)
potentialHeight : float = 200
preRegionLength = .3
preRegionScalar = 2
packetStandardDeviation : float = preRegionLength / preRegionScalar
packetStartX : float = 1 - ((preRegionScalar / 2) * packetStandardDeviation)
packetStartY : float = .5
packetWaveNumber : float = 13 * np.pi
barrierCount : int = 3
postRegionLength = .3
intraRegionLength : float = 1 - (postRegionLength + preRegionLength)
barrierWidth : float = intraRegionLength / barrierCount
constantPotentialLengthRatios : List[float] = [postRegionLength] \
        + ([barrierWidth] * barrierCount) \
        + [preRegionLength]
print("Point Count: ", pointCount)
print("Post Region Length: ", postRegionLength)
print("Intra Region Length: ", intraRegionLength)
print("Pre Region Length: ", preRegionLength)
print("Barrier Length: ", barrierWidth)
print("Total Length: ", intraRegionLength + postRegionLength + preRegionLength)
print("Total Length Ratios: ", np.sum(constantPotentialLengthRatios))
frameCount : int = 650
backend = "numpy"

Point Count:  50
Post Region Length:  0.3
Intra Region Length:  0.4
Pre Region Length:  0.3
Barrier Length:  0.13333333333333333
Total Length:  1.0
Total Length Ratios:  1.0


In [11]:
wavePacketFunction = lambda position : makeWavePacket(
            position, 
            packetStartX * length, 
            packetStartY * length, 
            spatialStep, 
            packetStandardDeviation, 
            packetWaveNumber, 
            math
        )

In [12]:
potentials : List[List[float]] = [
        [0, 1, 2 / 3, 1 / 3, 0], 
        [0, 1, 1, 1, 0]
        [0, 0, 0, 0, 0]
    ]

In [13]:
print(potentials[0])
potentials[0].reverse()
print(potentials[0])

[0, 0, 0, 0, 0]
[0, 0, 0, 0, 0]


In [14]:
profiles : List[SimulationProfile] = constantSimulationProfiles(
        wavePacketFunction, 
        spatialStep, 
        temporalStep, 
        length, 
        constantPotentialLengthRatios, 
        potentials, 
        potentialHeight, 
        pointCount, 
        True, 
        edgeBound = True
    )

In [15]:
constantPotentialLengthRatios

[0.3, 0.13333333333333333, 0.13333333333333333, 0.13333333333333333, 0.3]

In [16]:
allData, _ = recordConstantRegionSimulations(
        profiles, 
        frameCount, 
        caseName, 
        constantPotentialLengthRatios, 
        False, 
        constantRegionLabels = ["Post", "3rdStair", "2ndStair", "1stStair", "Pre"], 
        showBar = True, 
        showFPS = True, 
        showTotalTime = True
    )

[====================================================================================================]
[----------------------------------------------------------------------------------------------------]
Total Time:  4.368319348999876
Frames Per Second:  123.51499359955726
Saving Video of Post
Saving Video of 3rdStair
Saving Video of 2ndStair
Saving Video of 1stStair
Saving Video of Pre
[====================================================================================================]
[----------------------------------------------------------------------------------------------------]
Total Time:  3.748244502000034
Frames Per Second:  167.11704556425136
Saving Video of Post
Saving Video of 3rdStair
Saving Video of 2ndStair
Saving Video of 1stStair
Saving Video of Pre


In [17]:
pd.DataFrame(allData).to_csv(str(Path.cwd() / caseName / (caseName + ".csv")))